# Litterbot

Device von CPU auf GPU wechseln

In [ ]:
import torch
device = torch.device('cuda')

TRT optimierte Modelle laden.

> WICHTIG: Modelle müssen zuerst mit den Notebooks zum Datensammeln und Trainieren erstellt werden. Die Modelle mit der Endung .pth müssen im gleichen Ordner sein, wie dieses Notebook

In [ ]:
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) # well trained road following model

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth')) # well trained collision avoidance model

### Pre-Processing Funktion
1. Konvertieren von HWC-Layout zu CHW-Layout
2. Normalisieren mit denselben Parametern wie beim Training (Kamera liefert Werte im Bereich [0, 255] und die beim Training geladenen Bilder im Bereich [0, 1], so dass um 255,0 skaliert werden muss.
3. Übertragen der Daten vom CPU-Speicher in den GPU-Speicher.
4. Hinzufügen einer Batch-Dimension

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Starten der Kamera

In [1]:
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=10)

Erstellen der Roboter Instanz

In [3]:
import sys
import os
sys.path.append(os.path.abspath("/workspace/muellot_jetbot/jetbot"))

from robot2 import Robot2

robot = Robot2()

### Kontrollinterface für den Roboter

In [31]:
from IPython.display import display
import traitlets
from IPython.display import display
from ipywidgets import Layout, Label
import ipywidgets.widgets as widgets

image_widget = widgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

on_off_widget = widgets.ToggleButtons(options=['On', 'Off'], button_style='danger', description='On / Off', value='Off')
state_widget = widgets.ToggleButtons(options=['Manuell', 'Auto'], description='Camera', value='Manuell')
display(widgets.VBox([on_off_widget,state_widget]))

#Road Following sliders
network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_dgain_slider = widgets.FloatSlider(description='Steering kd', min=0.0, max=0.5, step=0.001, value=0.06, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.15, step=0.01, orientation='vertical')

#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(description='Blocked', min=0.0, max=1.0, orientation='horizontal')
stopduration_slider= widgets.IntSlider(description='Time for Stop', min=1, max=1000, step=1, value=10, orientation='horizontal') 
blocked_threshold= widgets.FloatSlider(description='Blocked Threshold', min=0, max=1.0, step=0.01, value=0.8, orientation='horizontal')


#steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (robot, 'steering_gain'))
#steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (robot, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (robot, 'steering'))
#throttle_slider_link = traitlets.link((throttle_slider, 'value'), (robot, 'throttle'))

lout=Layout(align_items='center', border='3px solid black', padding='3px')

auto_widgets = widgets.HBox([
    widgets.VBox([Label('Road Following Sliders'),
        network_output_slider,
        steering_gain_slider,
        steering_dgain_slider,
        steering_bias_slider, 
        steering_value_slider], 
        layout = lout
        ),
    throttle_slider,
    image_widget,
    widgets.VBox([Label('Collision Avoidance Sliders'),
        blocked_slider,
        stopduration_slider,
        blocked_threshold],
        layout = lout)],
    layout = lout
    )

display(auto_widgets)

In [ ]:
import cv2
import math

angle = 0.0
angle_last = 0.0
count_stops = 0
go_on = 1
stop_time = 10 # The number of frames to remain stopped
x = 0.0
y = 0.0
speed_value = throttle_slider.value

def execute(change):
    global angle, angle_last, blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global speed_value, steer_gain, steer_dgain, steer_bias
                
    steer_gain = steering_gain_slider.value
    steer_dgain = steering_dgain_slider.value
    steer_bias = steering_bias_slider.value
       
    image_preproc = preprocess(change['new']).to(device)
     
    #Collision Avoidance model:
    
    prob_blocked = float(F.softmax(model_trt_collision(image_preproc), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time=stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
            xy = model_trt(image_preproc).detach().float().cpu().numpy().flatten()        
            x = xy[0]            
            y = (0.5 - xy[1]) / 2.0
            network_output_slider.value = x
            speed_value = speed_control_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            x = 0.0 #set x steering to zero
            y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero
        else:
            go_on = 1
            count_stops = 0
            
    
    angle = math.atan2(x, y)        
    pid = angle * steer_gain + (angle - angle_last) * steer_dgain
    steering_value_slider.value = pid + steer_bias
    angle_last = angle
    robot.throttle = speed_value
    robot.steering = max(min(steering_value_slider.value, 1.0), -1.0) 

execute({'new': camera.value}) # call the function once to initialize

In [ ]:
camera.observe(execute, names='value') 

In [ ]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [8]:
camera.stop()